In [ ]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

In [ ]:
csv_file = 'C:/Users/Ashrit/Desktop/Long Short Hair Identification/Hair_length_model/hair_length_.csv'
data = pd.read_csv(csv_file, names=['filename', 'label'])

In [ ]:
# Paths to the images folder
image_folder = 'C:/Ashrit/Dataset_1/hairdata_1'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    validation_split=0.2  # Split 20% for validation
)

In [ ]:
# For training data
train_generator = datagen.flow_from_directory(
    'C:/Ashrit/Dataset_1/hairdata_1',  # Path to dataset directory
    target_size=(128, 128),  # Resize images to 224x224
    batch_size=32,
    class_mode='categorical',  # Or 'binary' if binary classification
    subset='training'  # Specify training subset
)


In [ ]:
# For validation data
val_generator = datagen.flow_from_directory(
    'C:/Ashrit/Dataset_1/hairdata_1',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Specify validation subset
)

In [ ]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # Binary classification (long/short)
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

In [ ]:
# Save the trained model
model.save('C:/Users/Ashrit/Desktop/Long Short Hair Identification/Models/hair_length_predict_model.keras')

print("Model training completed and saved as 'hair_length_predict_model.keras'.")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Get predictions on the validation set
val_generator.reset()  # Reset generator before predictions
predictions = model.predict(val_generator)
predicted_classes = np.argmax(predictions, axis=1)  # Get class with highest probability
true_classes = val_generator.classes  # True labels
class_labels = list(val_generator.class_indices.keys())  # Class labels


In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Classification Report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print("Classification Report:\n")
print(report)

In [ ]:
for layer in model.layers:
    print(f"Layer: {layer.name}")
    print("Weights:")
    print(layer.get_weights())  # This will output the weights and biases of the layer
    print()


In [ ]:
# Save weights only
model.save_weights('C:/Users/Ashrit/Desktop/Long Short Hair Identification/Models/hair_length.weights.h5')
print("Model weights saved.")


In [ ]:
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
non_trainable_params = np.sum([np.prod(v.shape) for v in model.non_trainable_weights])
total_params = trainable_params + non_trainable_params

print(f"Trainable Parameters: {trainable_params}")
print(f"Non-Trainable Parameters: {non_trainable_params}")
print(f"Total Parameters: {total_params}")
